In [1]:
import torch
import json
from PIL import Image
import requests
from tqdm import tqdm
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration

In [2]:
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(device)

cuda


In [4]:
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")

In [5]:
imagesPath = "/scratch/averma90/CLEVR_v1.0/images/val/"
quesPath = "/home/averma90/CSE576/github/CLEVR_v1.0/questions/CLEVR_val_questions.json"

jsonFile = open(quesPath, 'r')
questions  = json.load(jsonFile)

In [ ]:
for i in range(1,11):
    img_file = open(f"/home/averma90/CSE576/github/MLLM_Hallucinations/CLEVR_v1/datasetSplits/val_images_objectsNum{i}.txt", "r") 
    img_names = img_file.readlines()
    
    ansFile = open(f"/home/averma90/CSE576/github/CLEVR_v1.0/answers/val/num_{i}.txt", "a")
    
    for img_name in tqdm(img_names[:100]):
        image_ques = []
        image_ans = []
        count = 0
        img = Image.open(imagesPath+img_name[:-1]).convert("RGB")
        for q in questions["questions"]:
            if q["image_filename"] == img_name[:-1]:
                image_ques.append(q["question"])
                image_ans.append(q["answer"])
        for iqs in image_ques:
            prompt=iqs
            inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
            outputs = model.generate(
                **inputs,
                do_sample=False,
                num_beams=5,
                max_length=256,
                min_length=1,
                top_p=0.9,
                repetition_penalty=1.5,
                length_penalty=1.0,
                temperature=1,
            )
            generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
            # print(generated_text)
            ansFile.write(f"{prompt} Answer: {image_ans[count]}. Output: {generated_text}\n")
            # ansFile.write(prompt +" " + image_ans[count] + ": " + generated_text+"\n")
            count+=1
    ansFile.close()

0it [00:00, ?it/s]
0it [00:00, ?it/s]
 73%|███████▎  | 73/100 [14:20<05:50, 12.97s/it]